# Fine Tuning GPT-2 for Chexplanations 
The purpose of this notebook is to fine-tune OpenAI's [GPT-2](https://openai.com/api/) for the task of summarisation and sentence aggregation in the context of Chess commentary.

In [3]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2

Download GPT-2's 'medium' model. 

In [4]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 102Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.64Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 477Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001:  23%|███▊             | 320M/1.42G [00:09<00:33, 32.9Mit/s]


KeyboardInterrupt: ignored

Connect to google drive and retrieve data. 

In [6]:
from google.colab import files
gpt2.mount_gdrive()
gpt2.copy_file_from_gdrive('training.txt')

Mounted at /content/drive


In [19]:
# preview the dataset
with open('training.txt', 'r') as r:
    data = r.readlines()
    print("{} training examples.".format(len(data)))
    
    example_no, count = 10, 0
    for line in data:
      print(line)
      count += 1
      if count == example_no: 
        break
  

660 training examples.
ORIGINAL: You open with the Amar Opening. You bring the knight out.

PARAPHRASED: You bring the knight out, opening with the Amar Opening.

<|endoftext|>

ORIGINAL: You open with the Amar Opening. You bring the knight out.

PARAPHRASED: You open with the Amar opening by bringing the knight out.

<|endoftext|>

ORIGINAL: You open with the Amar Opening. You bring the knight out.

PARAPHRASED: By bringing the knight out you open with the Amar Opening.

<|endoftext|>

ORIGINAL: You open with the Amar Opening. You bring the knight out.



## Train the model 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(
    sess,
    dataset='training.txt',
    model_name='355M',
    steps=1000,
    restore_from='latest',
    overwrite=True,
    run_name='run1',
    print_every=100,
    sample_every=200,
    save_every=500
)

gpt2.copy_checkpoint_to_gdrive(run_name='run1')

## Test the model 

In [7]:
# copy checkpoint saved model
gpt2.copy_checkpoint_from_gdrive(run_name='run1')
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1013
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1013


In [8]:
test_text = "ORIGINAL: Your opponent responds with the Indian Defense. Stockfish brings the knight out. You may want to try e2e4, f2f3 or g2g3. Playing e2e4 leads to the Omega Gambit. Playing f2f3 leads to the Paleface Attack. Playing g2g3 leads to the Tartakower Attack."

gpt2.generate(
    sess,
    length=100,
    temperature=0.8,
    prefix=test_text,
    truncate='<|endoftext|>',
    nsamples=5,
    batch_size=5
)

ORIGINAL: Your opponent responds with the Indian Defense. Stockfish brings the knight out. You may want to try e2e4, f2f3 or g2g3. Playing e2e4 leads to the Omega Gambit. Playing f2f3 leads to the Paleface Attack. Playing g2g3 leads to the Tartakower Attack.
PARAPHRASED: The Indian Defense is Black's response by bringing the knight out. You may try e2e4, f2f3, or g2g3. The Paleface Attack is triggered when you play e2e4. The Tartakower Attack occurs when you play f2f3. The Paleface Attack is triggered when you play g2g3.

ORIGINAL: Your opponent responds with the Indian Defense. Stockfish brings the knight out. You may want to try e2e4, f2f3 or g2g3. Playing e2e4 leads to the Omega Gambit. Playing f2f3 leads to the Paleface Attack. Playing g2g3 leads to the Tartakower Attack.
PARAPHRASED: The Indian Defense is Black's response by bringing the knight out. You may try e2e4, f2f3, or g2g3. The Paleface Attack is triggered when you play e2e4. The Tartakower Attack occurs when you play f2f3